#Curso Machine Learning: Validação de Dados
Esse é um curso disponibilizado pela [Alura](https://cursos.alura.com.br/course/machine-learning-validando-modelos).

#Validação Cruzada e Aleatoriedade Inicial - Aula 01

Importando as bibliotecas Necessárias

In [0]:
#Bibliotecas Básicas:
import pandas as pd
import numpy as np

#Divisão do DataSet
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

#Cross Validation:
from sklearn.model_selection import cross_validate

#Importando o Kfold (aula 02):
from sklearn.model_selection import KFold

#Importando o StratifiedKfold (aula 03):
from sklearn.model_selection import StratifiedKFold

#Importando o GroupKFold (aula 04):
from sklearn.model_selection import GroupKFold

#Importando o StandardScaler (aula 05):
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

Importando o arquivo para analise

In [44]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


Divisão do DataSet

In [45]:
x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']
SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print('Treinaremos com %d e testaremos com %d elementos' % (len(treino_x), len(teste_x)))

Treinaremos com 7500 e testaremos com 2500 elementos


In [46]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print('A Acurácia do Dummy Stratified foi %.2f%%' % acuracia)

A Acurácia do Dummy Stratified foi 50.96%


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [47]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth = 2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('A Acuracia foi de {}%'.format(acuracia))

A Acuracia foi de 71.92%


K-Fold: Corta em k partes

In [48]:
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score= False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('A Acuracia foi de {:.2f} e {:.2f}'.format(media - 2 * desvio_padrao, media + 2 * desvio_padrao))

A Acuracia foi de 0.75 e 0.76


#Kfold e Aleatoriedade

In [0]:
def imprime_resultados(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print('Acurácia Média {:.2f}'.format(media*100))
  print('O Intervalo de Acuracia foi de {:.2f} e {:.2f}'.format(media - 2 * desvio_padrao, media + 2 * desvio_padrao))

In [50]:
#Sem Shuffle, Apenas com KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score= False)
imprime_resultados(results)

Acurácia Média 75.78
O Intervalo de Acuracia foi de 0.74 e 0.77


In [51]:
#Com Shuffle
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score= False)
imprime_resultados(results)

Acurácia Média 75.76
O Intervalo de Acuracia foi de 0.73 e 0.78


#Estratificação com Validação Cruzada - Aula 03

##Simulação de situação horrível de azar
Pode ser "azar" ccomo pode ser uma proporção de exemplos desbalanceado entre as classes

In [52]:
dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [53]:
#Sem Shuffle, Apenas com KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score= False)
imprime_resultados(results)

Acurácia Média 57.84
O Intervalo de Acuracia foi de 0.34 e 0.81


In [54]:
#Com Shuffle
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score= False)
imprime_resultados(results)

Acurácia Média 75.78
O Intervalo de Acuracia foi de 0.72 e 0.79


In [55]:
#Com Stratified Kfold e Shuffle
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score= False)
imprime_resultados(results)

Acurácia Média 75.78
O Intervalo de Acuracia foi de 0.74 e 0.78


#Dados Agrupáveis - Aula 04

In [56]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size = 10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [57]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(results)

Acurácia Média 75.78
O Intervalo de Acuracia foi de 0.74 e 0.78


#Pipeline de treino e validação - Aula 05

#Cross Validation com StandardScaler

In [58]:
scaler = StandardScaler()

scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('A Acuracia foi de {}%'.format(acuracia))


A Acuracia foi de 74.4%


In [59]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(results)

Acurácia Média 77.27
O Intervalo de Acuracia foi de 0.74 e 0.80


In [0]:
scaler = StandardScaler()

scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [61]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(results)

Acurácia Média 76.70
O Intervalo de Acuracia foi de 0.74 e 0.79


In [63]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(results)

Acurácia Média 76.68
O Intervalo de Acuracia foi de 0.74 e 0.79
